In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel("aula_3.xlsx")
df.shape

(58, 12)

In [ ]:
df.head()

,nome,data_ingresso,curso,cpf,estado,cidade,endereco,numero,pagamento,telefone,vendedor,valor_curso
0,abelar henrique,2023-03-05,engenharia dados,68740550196,PR,curitiba,Rua da Penha,3300,a vista,866862753,paulo,60
1,abelar henrique,2023-03-05,engenharia dados,68740550196,PR,curitiba,Rua da Penha,3300,a vista,274845545,paulo,60
2,adriano gambetta,2023-03-06,engenharia dados,34146481132,PR,curitiba,Rua Henrique Soares,7118,credito,273807947,adriana,60
3,amanda moulin,2023-03-07,engenharia dados,12728282203,PR,curitiba,Rua Leodécio Santiago de Souza,6082,credito,872235320,paulo,60
4,amanda moulin,2023-03-07,engenharia dados,12728282203,PR,curitiba,Rua Leodécio Santiago de Souza,6082,credito,911955261,paulo,60


In [ ]:
df["primeiro_nome"] = df["nome"].apply(lambda x: x.split(" ")[0])
df["ultimo_nome"] = df["nome"].apply(lambda x: x.split(" ")[1])

In [ ]:
df.drop(columns = "nome", inplace = True)

In [ ]:
# valor_cursos = {
#     "engenharia dados": 60,
#     "ciencia dados": 70,
#     "analise dados": 50,
#     "arquiteto dados": 80
# }

# df.loc[:, "valor_curso"] = df["curso"].map(valor_cursos)

In [ ]:
# df.to_excel("base_curso.xlsx", index = False)

In [ ]:
df.loc[:, "telefone"] = df["telefone"].apply(str)
df.loc[:, "cpf"] = df["cpf"].apply(lambda x: str(x).zfill(11))

# Emancipando a tabela de alunos

In [ ]:
# selecionamos informações associadas aos alunos
alunos = df[["primeiro_nome", "ultimo_nome", "data_ingresso","cpf", "endereco", "numero", "cidade", "estado"]].drop_duplicates()

# criamos um id_aluno
alunos.loc[:, "id_aluno"] = range(1, len(alunos) + 1)

In [ ]:
alunos.head()

,primeiro_nome,ultimo_nome,data_ingresso,cpf,endereco,numero,cidade,estado,id_aluno
0,abelar,henrique,2023-03-05,68740550196,Rua da Penha,3300,curitiba,PR,1
2,adriano,gambetta,2023-03-06,34146481132,Rua Henrique Soares,7118,curitiba,PR,2
3,amanda,moulin,2023-03-07,12728282203,Rua Leodécio Santiago de Souza,6082,curitiba,PR,3
5,álvaro,fonseca,2023-03-08,43764914607,Rua Pitica,7924,curitiba,PR,4
7,andre,melchior,2023-03-09,23228327392,Rua Camilo Paula,4160,Sao paulo,SP,5


# Organizando os telefones

In [ ]:
# selecionando os telefones
tel = df[["cpf", "telefone"]]
tel.loc[(~tel.cpf.duplicated(keep = "first")), "n_telefone"] = "telefone_a"
tel.loc[(tel.cpf.duplicated(keep = "first")), "n_telefone"] = "telefone_b"

C:\Users\r-omd\AppData\Local\Temp\ipykernel_38068\668768538.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tel.loc[(~tel.cpf.duplicated(keep = "first")), "n_telefone"] = "telefone_a"


In [ ]:
tel.head()

,cpf,telefone,n_telefone
0,68740550196,866862753,telefone_a
1,68740550196,274845545,telefone_b
2,34146481132,273807947,telefone_a
3,12728282203,872235320,telefone_a
4,12728282203,911955261,telefone_b


In [ ]:
# pivoteando os valores de telefone
tel_1 = tel.pivot_table(index = "cpf", columns = "n_telefone", values = "telefone").reset_index()
tel_1.head()

n_telefone,cpf,telefone_a,telefone_b
0,02456915690,538617705.0,NaN
1,05494797137,845459713.0,615675266.0
2,05990749922,708729599.0,NaN
3,07562543599,326789195.0,NaN
4,11423428746,730517155.0,NaN


In [ ]:
# fazendo um join com os valores dos telefones
alunos_1 = alunos.merge(tel_1, on = "cpf", how = "left")
alunos_1.shape

(34, 11)

In [ ]:
alunos_1.head()

,primeiro_nome,ultimo_nome,data_ingresso,cpf,endereco,numero,cidade,estado,id_aluno,telefone_a,telefone_b
0,abelar,henrique,2023-03-05,68740550196,Rua da Penha,3300,curitiba,PR,1,866862753.0,274845545.0
1,adriano,gambetta,2023-03-06,34146481132,Rua Henrique Soares,7118,curitiba,PR,2,273807947.0,NaN
2,amanda,moulin,2023-03-07,12728282203,Rua Leodécio Santiago de Souza,6082,curitiba,PR,3,872235320.0,911955261.0
3,álvaro,fonseca,2023-03-08,43764914607,Rua Pitica,7924,curitiba,PR,4,938535574.0,262020598.0
4,andre,melchior,2023-03-09,23228327392,Rua Camilo Paula,4160,Sao paulo,SP,5,189494206.0,323561018.0


# Emancipando a tabela das cidades

In [ ]:
# selecionando os valores associados a cidade (dependencia transitiva)
cidades = alunos_1[["cidade", "estado"]].drop_duplicates()
cidades.head()

,cidade,estado
0,curitiba,PR
4,Sao paulo,SP
8,salvador,BA
12,jequié,BA
19,tiradentes,mg


In [ ]:
# tratando os valores das cidades
cidades.loc[:,"cidade"] = cidades["cidade"].str.strip().str.lower()
cidades = cidades.drop_duplicates()

In [ ]:
cidades

,cidade,estado
0,curitiba,PR
4,sao paulo,SP
8,salvador,BA
12,jequié,BA
19,tiradentes,mg
20,uberlandia,MG
22,cogonhas,MG
23,congonhas,MG
25,porto velho,RO
28,jaru,RO


In [ ]:
# arrumando os valores das cidades estranhas
cidades.loc[:, "cidade"] = cidades["cidade"].str.replace("cwb", "curitiba").str.replace("cogonhas", "congonhas")
cidades = cidades.drop_duplicates()

In [ ]:
# criando id_cidade
cidades.loc[:, "id_cidade"] = range(1, len(cidades) + 1)

C:\Users\r-omd\AppData\Local\Temp\ipykernel_38068\4071042687.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cidades.loc[:, "id_cidade"] = range(1, len(cidades) + 1)


In [ ]:
cidades

,cidade,estado,id_cidade
0,curitiba,PR,1
4,sao paulo,SP,2
8,salvador,BA,3
12,jequié,BA,4
19,tiradentes,mg,5
20,uberlandia,MG,6
22,congonhas,MG,7
25,porto velho,RO,8
28,jaru,RO,9
30,alto paraíso,RO,10


In [ ]:
# tratando os valores associados a cidade
alunos_1.loc[:, "cidade"] = alunos_1["cidade"].str.strip().str.lower().str.replace("cwb", "curitiba").str.replace("cogonhas", "congonhas")

In [ ]:
# fazendo um join com o id_cidade
alunos_2 = alunos_1.merge(cidades, on = ["cidade", "estado"], how = "left")
alunos_2.shape

(34, 12)

In [ ]:
alunos_2.head()

,primeiro_nome,ultimo_nome,data_ingresso,cpf,endereco,numero,cidade,estado,id_aluno,telefone_a,telefone_b,id_cidade
0,abelar,henrique,2023-03-05,68740550196,Rua da Penha,3300,curitiba,PR,1,866862753.0,274845545.0,1
1,adriano,gambetta,2023-03-06,34146481132,Rua Henrique Soares,7118,curitiba,PR,2,273807947.0,NaN,1
2,amanda,moulin,2023-03-07,12728282203,Rua Leodécio Santiago de Souza,6082,curitiba,PR,3,872235320.0,911955261.0,1
3,álvaro,fonseca,2023-03-08,43764914607,Rua Pitica,7924,curitiba,PR,4,938535574.0,262020598.0,1
4,andre,melchior,2023-03-09,23228327392,Rua Camilo Paula,4160,sao paulo,SP,5,189494206.0,323561018.0,2


In [ ]:
# removendo a dependencia transitiva da cidade (utilizando o id_cidade)

alunos_3 = alunos_2[
        [
            "id_aluno",
            "cpf",
            "primeiro_nome",
            "ultimo_nome",
            "data_ingresso",
            "id_cidade",
            "endereco",
            "numero",
            "telefone_a",
            "telefone_b"
        ]
    ]

alunos_3.shape

(34, 10)

In [ ]:
alunos_3.loc[:, "data_ingresso"] = alunos_3["data_ingresso"].dt.strftime("%Y-%m-%d")
alunos_3["telefone_b"].fillna(0, inplace=True)
# alunos_3.loc[:, "telefone_a"] = alunos_3["telefone_a"].apply(lambda x: x.replace(".0",""))
# alunos_3.loc[:, "telefone_b"] = alunos_3["telefone_b"].apply(lambda x: x.replace(".0",""))

alunos_3.loc[:, "telefone_a"] = alunos_3["telefone_a"].apply(lambda x: str(int(x)))
alunos_3.loc[:, "telefone_b"] = alunos_3["telefone_b"].apply(lambda x: str(int(x)))

C:\Users\r-omd\AppData\Local\Temp\ipykernel_38068\3094181176.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alunos_3.loc[:, "data_ingresso"] = alunos_3["data_ingresso"].dt.strftime("%Y-%m-%d")
C:\Users\r-omd\AppData\Local\Temp\ipykernel_38068\3094181176.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alunos_3["telefone_b"].fillna(0, inplace=True)
C:\Users\r-omd\AppData\Local\Temp\ipykernel_38068\3094181176.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

# Emancipando cursos

In [ ]:
# criando o id_curso

cursos = df[["curso", "valor_curso"]].drop_duplicates().reset_index(drop = True)
cursos.loc[:, "id_curso"] = range(1, len(cursos) + 1)
cursos

,curso,valor_curso,id_curso
0,engenharia dados,60,1
1,ciencia dados,70,2
2,analise dados,50,3
3,arquiteto dados,80,4


# Emancipando pagamento

In [ ]:
# criando o id_pagamento

pagamento = df[["pagamento"]].drop_duplicates().reset_index(drop = True)
pagamento.loc[:, "id_pagamento"] = range(1, len(pagamento) + 1)
pagamento

,pagamento,id_pagamento
0,a vista,1
1,credito,2
2,pix,3
3,carnê,4
4,crediário,5


# Emancipando vendedores

In [ ]:
# criando o id_vendedor

vendedores = df[["vendedor"]].drop_duplicates().reset_index(drop = True)
vendedores.loc[:, "id_vendedor"] = range(1, len(vendedores) + 1)
vendedores

,vendedor,id_vendedor
0,paulo,1
1,adriana,2
2,nadir,3


# Emancipando tabela de transações

In [ ]:
# organizando a tabela principal com os registros de transações

df_1 = df.drop(columns = ["primeiro_nome", "ultimo_nome", "data_ingresso", "estado", "cidade", "endereco", "numero", "telefone"])
df_2 = df_1.drop_duplicates()
df_2.shape

(34, 5)

In [ ]:
df_2.head()

,curso,cpf,pagamento,vendedor,valor_curso
0,engenharia dados,68740550196,a vista,paulo,60
2,engenharia dados,34146481132,credito,adriana,60
3,engenharia dados,12728282203,credito,paulo,60
5,engenharia dados,43764914607,a vista,adriana,60
7,engenharia dados,23228327392,pix,paulo,60


In [ ]:
# fazendo o join com todas as PRIMARY KEYS criadas

df_3 = df_2.merge(cursos, on = "curso", how = 'left')
df_3 = df_3.merge(alunos_3[["cpf", "id_aluno"]], on = 'cpf', how = "left")
df_3 = df_3.merge(pagamento, on = "pagamento", how = "left")
df_3 = df_3.merge(vendedores, on = "vendedor", how = "left")

In [ ]:
# criando um id_transacao

df_3.loc[:, "id_transacao"] = range(1, len(df_3) + 1)

In [ ]:
# selecionando apenas as PRIMARY e FOREIGN KEY para o registro das transacões
transacoes = df_3[["id_transacao","id_aluno", "id_curso", "id_vendedor", "id_pagamento"]]

In [ ]:
transacoes.head()

,id_transacao,id_aluno,id_curso,id_vendedor,id_pagamento
0,1,1,1,1,1
1,2,2,1,2,2
2,3,3,1,1,2
3,4,4,1,2,1
4,5,5,1,1,3


# Como ficou ao final?

In [ ]:
# tabela apenas com os valores dos alunos
alunos_3.head()

,id_aluno,cpf,primeiro_nome,ultimo_nome,data_ingresso,id_cidade,endereco,numero,telefone_a,telefone_b
0,1,68740550196,abelar,henrique,2023-03-05,1,Rua da Penha,3300,866862753,274845545
1,2,34146481132,adriano,gambetta,2023-03-06,1,Rua Henrique Soares,7118,273807947,0
2,3,12728282203,amanda,moulin,2023-03-07,1,Rua Leodécio Santiago de Souza,6082,872235320,911955261
3,4,43764914607,álvaro,fonseca,2023-03-08,1,Rua Pitica,7924,938535574,262020598
4,5,23228327392,andre,melchior,2023-03-09,2,Rua Camilo Paula,4160,189494206,323561018


In [ ]:
# tabela apenas com os valores das cidades
cidades = cidades[["id_cidade", 'cidade', "estado"]]
cidades.head()

,id_cidade,cidade,estado
0,1,curitiba,PR
4,2,sao paulo,SP
8,3,salvador,BA
12,4,jequié,BA
19,5,tiradentes,mg


In [ ]:
# tabela apenas com os valores de cursos
cursos = cursos[["id_curso", "curso", "valor_curso"]]
cursos.head()

,id_curso,curso,valor_curso
0,1,engenharia dados,60
1,2,ciencia dados,70
2,3,analise dados,50
3,4,arquiteto dados,80


In [ ]:
# tabela apenas com os valores de vendedores
vendedores = vendedores[["id_vendedor","vendedor"]]
vendedores.head()

,id_vendedor,vendedor
0,1,paulo
1,2,adriana
2,3,nadir


In [ ]:
# tabela apenas com os valores de pagamento
pagamento = pagamento[["id_pagamento", "pagamento"]]
pagamento.head()

,id_pagamento,pagamento
0,1,a vista
1,2,credito
2,3,pix
3,4,carnê
4,5,crediário


In [ ]:
# tabela apenas com os valores de transacoes
transacoes.head()

,id_transacao,id_aluno,id_curso,id_vendedor,id_pagamento
0,1,1,1,1,1
1,2,2,1,2,2
2,3,3,1,1,2
3,4,4,1,2,1
4,5,5,1,1,3


In [ ]:
pasta = "base_curso"
alunos_3.to_csv(f"{pasta}/alunos.csv", index = False)
cidades.to_csv(f"{pasta}/cidades.csv", index = False)
cursos.to_csv(f"{pasta}/cursos.csv", index = False)
vendedores.to_csv(f"{pasta}/vendedores.csv", index = False)
pagamento.to_csv(f"{pasta}/pagamento.csv", index = False)
transacoes.to_csv(f"{pasta}/transacoes.csv", index = False)

# DAQUI PRA BAIXO NÃO PRECISA RODAR

# CRIANDO INSERÇÕES

In [ ]:
def cria_insert_values(df, nome_df):

    lista_sem_pk = df.columns.tolist()[1:]

    texto_valores = ""
    valores = df[lista_sem_pk].to_records(index = False)
    for i in valores:
        texto_valores += str(i) + "\n, "

    texto_valores += " "

    texto_valores = texto_valores.replace("\n,  ", ";").replace(",)", ")")

    texto_colunas = "("
    for i in lista_sem_pk:
        texto_colunas += i + ", "

    texto_colunas += " )"

    texto_colunas = texto_colunas.replace(",  ", "")

    print(f"INSERT INTO ada_teste.{nome_df} {texto_colunas} VALUES {texto_valores}")

In [ ]:
cria_insert_values(cursos, "cursos")

INSERT INTO ada_teste.cursos (curso, valor_curso) VALUES ('engenharia dados', 60)
, ('ciencia dados', 70)
, ('analise dados', 50)
, ('arquiteto dados', 80);


In [ ]:
cria_insert_values(pagamento, "pagamentos")

INSERT INTO ada_teste.pagamentos (pagamento) VALUES ('a vista')
, ('credito')
, ('pix')
, ('carnê')
, ('crediário');


In [ ]:
cria_insert_values(cidades, "cidades")

INSERT INTO ada_teste.cidades (cidade, estado) VALUES ('curitiba', 'PR')
, ('sao paulo', 'SP')
, ('salvador', 'BA')
, ('jequié', 'BA')
, ('tiradentes', 'mg')
, ('uberlandia', 'MG')
, ('congonhas', 'MG')
, ('porto velho', 'RO')
, ('jaru', 'RO')
, ('alto paraíso', 'RO')
, ('ouro preto', 'MG');


In [ ]:
cria_insert_values(vendedores, "vendedores")

INSERT INTO ada_teste.vendedores (vendedor) VALUES ('paulo')
, ('adriana')
, ('nadir');


In [ ]:
cria_insert_values(alunos_3, "alunos")

INSERT INTO ada_teste.alunos (cpf, primeiro_nome, ultimo_nome, data_ingresso, id_cidade, endereco, numero, telefone_a, telefone_b) VALUES ('68740550196', 'abelar', 'henrique', '2023-03-05', 1, 'Rua da Penha', 3300, '866862753', '274845545')
, ('34146481132', 'adriano', 'gambetta', '2023-03-06', 1, 'Rua Henrique Soares', 7118, '273807947', '0')
, ('12728282203', 'amanda', 'moulin', '2023-03-07', 1, 'Rua Leodécio Santiago de Souza', 6082, '872235320', '911955261')
, ('43764914607', 'álvaro', 'fonseca', '2023-03-08', 1, 'Rua Pitica', 7924, '938535574', '262020598')
, ('23228327392', 'andre', 'melchior', '2023-03-09', 2, 'Rua Camilo Paula', 4160, '189494206', '323561018')
, ('76360071693', 'antonio', 'zanutto', '2023-03-10', 2, '1ª Travessa São Francisco', 8932, '379611062', '919134470')
, ('66346021768', 'carolina', 'yumi', '2023-03-11', 2, 'Quadra SHPS Quadra 603 Conjunto 96A', 5733, '422505705', '0')
, ('05494797137', 'christyan', 'costa', '2023-03-12', 2, 'Quadra 504 Sul Alameda 11', 8

In [ ]:
cria_insert_values(transacoes, "transacoes")

INSERT INTO ada_teste.transacoes (id_aluno, id_curso, id_vendedor, id_pagamento) VALUES (1, 1, 1, 1)
, (2, 1, 2, 2)
, (3, 1, 1, 2)
, (4, 1, 2, 1)
, (5, 1, 1, 3)
, (6, 1, 1, 3)
, (7, 1, 2, 1)
, (8, 1, 2, 1)
, (9, 2, 1, 4)
, (10, 2, 2, 4)
, (11, 2, 2, 2)
, (12, 2, 2, 1)
, (13, 2, 2, 1)
, (14, 2, 1, 1)
, (15, 2, 1, 5)
, (16, 2, 1, 2)
, (17, 2, 1, 2)
, (18, 2, 3, 3)
, (19, 2, 3, 5)
, (20, 2, 3, 3)
, (21, 2, 1, 1)
, (22, 3, 3, 5)
, (23, 3, 3, 5)
, (24, 3, 3, 4)
, (25, 3, 3, 4)
, (26, 3, 1, 5)
, (27, 3, 1, 2)
, (28, 3, 1, 2)
, (29, 3, 1, 5)
, (30, 4, 2, 5)
, (31, 4, 2, 2)
, (32, 4, 2, 2)
, (33, 4, 2, 2)
, (34, 4, 2, 5);


In [ ]:
%%sql

CREATE SCHEMA ada_teste;

CREATE TABLE ada_teste.cidades (
    id_cidade SERIAL PRIMARY KEY,
    cidade VARCHAR(100),
    estado CHAR(2)
);

CREATE TABLE ada_teste.alunos (
    id_aluno SERIAL PRIMARY KEY,
    id_cidade INTEGER,
    cpf CHAR(11),
    primeiro_nome VARCHAR(100),
    ultimo_nome VARCHAR(100),
    data_ingresso DATE,
    endereco VARCHAR(100),
    numero INTEGER,
    telefone_a VARCHAR(10),
    telefone_b VARCHAR(10),
    FOREIGN KEY (id_cidade) REFERENCES ada_teste.cidades (id_cidade)
);

CREATE TABLE ada_teste.cursos (
    id_curso SERIAL PRIMARY KEY,
    curso VARCHAR(20),
    valor_curso FLOAT
);

CREATE TABLE ada_teste.pagamentos (
    id_pagamento SERIAL PRIMARY KEY,
    pagamento VARCHAR(10)
);

CREATE TABLE ada_teste.vendedores (
    id_vendedor SERIAL PRIMARY KEY,
    vendedor VARCHAR(50)
);

CREATE TABLE ada_teste.transacoes (
    id_transacao SERIAL PRIMARY KEY,
    id_aluno INTEGER,
    id_curso INTEGER,
    id_vendedor INTEGER,
    id_pagamento INTEGER,
    FOREIGN KEY (id_aluno) REFERENCES ada_teste.alunos (id_aluno),
    FOREIGN KEY (id_curso) REFERENCES ada_teste.cidades (id_cidade),
    FOREIGN KEY (id_vendedor) REFERENCES ada_teste.vendedores (id_vendedor),
    FOREIGN KEY (id_pagamento) REFERENCES ada_teste.pagamentos (id_pagamento)
);

UsageError: Cell magic `%%sql` not found.
